In [1]:
from util import *

In [2]:
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  
no_of_samples=50000
lines = [lines[i] for i in range(0,no_of_samples)]

In [3]:
userid = []
rating = []
docs = []
business = []
i = 0
j = 0

error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        #d = clean(l[3]).split()
        #kmers = getKmers(d)
        #d.extend(kmers)
        docs.append(l[3])
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)
        print e

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)



Training Data: Number of lines processed: 50000
Training Data: Length of userid array: 50000
Training Data: Length of rating array: 50000
Training Data: Length of docs array: 50000
Training Data: Length of business array: 50000
Training Data: Number of exceptions encountered: 0


In [4]:
def transformRating(rating):
    transformed=list()
    for r in rating:
        if(r<3.5):
            transformed.append(0)
        else:
            transformed.append(1)
    return transformed
    

transformRatings=transformRating(rating)
    

In [5]:
print len(transformRatings)
print transformRatings[0:10]

50000
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [6]:





from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000

data_samples = docs[:n_samples]

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


Extracting tf-idf features for NMF...
done in 9.042s.


In [7]:
from sklearn.linear_model import LogisticRegression

all_text_train = docs[:45000]
all_text_test = docs[45000:]



sentimentXtrain = tfidf_vectorizer.fit_transform(all_text_train)
sentimentXtest = tfidf_vectorizer.transform(all_text_test)


sentimentYtrain = transformRatings[:45000]
sentimentYtest = transformRatings[45000:]


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
import pandas as pd

In [17]:


classifier = MultinomialNB()
clf = classifier.fit(sentimentXtrain, sentimentYtrain)
preds = clf.predict(sentimentXtest)


precision = metrics.precision_score(sentimentYtest, preds)
recall = metrics.recall_score(sentimentYtest, preds)
f1 = metrics.f1_score(sentimentYtest, preds)
accuracy = accuracy_score(sentimentYtest, preds)
report = classification_report(sentimentYtest, preds)


In [18]:
print report

             precision    recall  f1-score   support

          1       1.00      1.00      1.00      5000

avg / total       1.00      1.00      1.00      5000

